In [1]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import split, col, when, mean, isnan, lit
import base64
import os

# Temporary
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
key_file = open("/home/bda_crypto_busters/repos/BigDataAnalytics/2_data_preprocessing/crypto/stream/crypto-busting-375023-6722d6967eca.json", "rb")
key = base64.b64encode(key_file.read())
key = key.decode("utf-8")

In [3]:
spark = SparkSession.builder \
    .master("yarn") \
    .appName("Pub/Sub Lite to Spark Streaming") \
    .config("spark.jars", "/home/bda_crypto_busters/repos/BigDataAnalytics/2_data_preprocessing/crypto/stream/pubsublite-spark-sql-streaming-LATEST-with-dependencies.jar") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/21 17:18:51 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/01/21 17:18:51 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/01/21 17:18:52 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/01/21 17:18:52 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


### Reading streaming data

In [4]:
project_number = 1072423212419
location = "europe-central2"
subscription_id = "bda-coinbase-sub-lite"

In [5]:
df = (
    spark.readStream.format("pubsublite")
    .option(
        "pubsublite.subscription",
        f"projects/{project_number}/locations/{location}/subscriptions/{subscription_id}",
    )
    .option("gcp.credentials.key", key)
    .load()
)

In [5]:
# w tej kolumnie powinny być dane
# df = df.withColumn('data', df.data.cast(StringType())).select('data')

In [6]:
df = df.withColumn('data', df.data.cast(StringType()))

In [7]:
# Print
res = df.writeStream \
    .format("console") \
    .outputMode("append") \
    .start()

23/01/21 17:19:25 WARN org.apache.spark.sql.streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4dc01046-9aa9-4a66-9bd9-4eb2b951e3f2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/21 17:19:25 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/01/21 17:19:48 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/01/21 17:20:03 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient res